In [1]:
!pip install pandas fiona shapely pyproj rtree
!pip install geopandas

In [2]:
pip install pandas-bokeh

Note: you may need to restart the kernel to use updated packages.


In [3]:
import geopandas as gpd
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [4]:
defibrillateurs = pd.read_csv("defibrillateur.csv")
police = pd.read_csv("police.csv")
hopitaux = pd.read_csv("hopitaux.csv")
camping = pd.read_csv("camping.csv")
departements = pd.read_csv("departements.csv")
communes = pd.read_csv("communes.csv")
#mobile = pd.read_csv("mobile.csv")
sans_mobile = pd.read_csv("sans_mobile.csv")

In [5]:
m = communes.plot_bokeh(line_width = 0.1, fill_alpha = 0, legend = "Communes", title = "Sites de Camping en Ile-de-France", show_figure = False)
m.width = 1000
m.height = 700
departements.plot_bokeh(figure = m, line_color = "black", fill_alpha = 0.3, color = "blue", legend = "Départements", show_figure = False)
sans_mobile.plot_bokeh(figure = m, legend = "Pas de Couverture Mobile", color = "grey", line_width = 0.5, show_figure = False)
defibrillateurs.plot_bokeh(figure = m, legend = "Défibrillateurs", size = 0.5, color = "green", show_figure = False)
police.plot_bokeh(figure = m, color = "blue", size = 2, legend = "Police/Gendarmerie", hovertool_columns = ["Nom"], show_figure = False)
hopitaux.plot_bokeh(figure = m, color = "yellow", size = 2, legend = "Hôpitaux/Cliniques",hovertool_columns = ["Nom"], show_figure = False)
camping.plot_bokeh(figure = m, size = 5, legend = "Campings (classés)", category = "Classement", colormap = "Reds", hovertool_columns = ["Nom", "Adresse", "Emplacements", "Classement", "Surface"])

AttributeError: unexpected attribute 'fill_alpha' to Line, similar attributes are line_alpha

In [ ]:
#visualiser les défibrillateurs a moins de 50m des sites de camping?
#visualiser les sites sans couverture mobile?
#footnote :ecrire ce que veut dire les etoiles des sites veulent dire (connexion Internet,  permanence 24h/24) en bas du graphique ou en haut